# Exemplu de Bază AutoGen

În acest exemplu de cod, veți utiliza cadrul AI [AutoGen](https://aka.ms/ai-agents/autogen) pentru a crea un agent de bază.

Scopul acestui exemplu este de a vă arăta pașii pe care îi vom folosi ulterior în exemplele suplimentare de cod atunci când implementăm diferitele tipare agentice.


## Importați pachetele Python necesare


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## Crearea Clientului

În acest exemplu, vom folosi [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) pentru acces la LLM.

`model` este definit ca `gpt-4o-mini`. Încercați să schimbați modelul cu un alt model disponibil pe piața GitHub Models pentru a vedea rezultate diferite.

Ca un test rapid, vom rula doar un prompt simplu - `Care este capitala Franței`.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definirea Agentului

Acum că am configurat `client` și am confirmat că funcționează, să creăm un `AssistantAgent`. Fiecare agent poate avea atribuit:  
**name** - Un nume scurt care va fi util pentru a-l referi în fluxurile cu mai mulți agenți.  
**model_client** - Clientul pe care l-ai creat în pasul anterior.  
**tools** - Instrumentele disponibile pe care Agentul le poate folosi pentru a finaliza o sarcină.  
**system_message** - Metapromptul care definește sarcina, comportamentul și tonul LLM-ului.  

Poți modifica mesajul sistemului pentru a vedea cum răspunde LLM-ul. Vom aborda `tools` în Lecția #4.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## Rulează Agentul

Funcția de mai jos va rula agentul. Folosim metoda `on_message` pentru a actualiza starea Agentului cu noul mesaj.

În acest caz, actualizăm starea cu un nou mesaj de la utilizator, care este `"Planifică-mi o vacanță grozavă și însorită"`.

Poți schimba conținutul mesajului pentru a vedea cum răspunde LLM diferit.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
